##### 4_featureEngineering

This notebook takes the csv that was compiled from the notebook 3_compileDischargesHospitalCensusData and does further processing on it such as: 
- Dealing with Categorical Variables

- Feature Engineering

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
aprHospFeat = pd.read_csv('dataFiles/compiledDischargesHospitalFeaturesCensus.csv', low_memory=False)

In [3]:
aprHospFeat.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['Unnamed: 0', 'Health Service Area', 'Facility Id', 'Length of Stay',
       'Type of Admission', 'APR DRG Code', 'APR DRG Description',
       'APR Severity of Illness Code', 'Payment Typology 1',
       'Payment Typology 2', 'Payment Typology 3',
       'Emergency Department Indicator', 'Total Charges', 'Total Costs', 'yr',
       'Provider ID', 'Hospital Ownership', 'Hospital overall rating',
       'Mortality national comparison', 'Safety of care national comparison',
       'Readmission national comparison',
       'Patient experience national comparison',
       'Effectiveness of care national comparison',
       'Timeliness of care national comparison',
       'Efficient use of medical imaging national comparison', 'lat', 'long',
       'lonlat', 'geometry', 'fte_employees_on_payroll', 'number_of_beds',
       'number_of_interns_and', 'rural_versus_urban', 'puma',
       'totalPopulation', 'totalInsured', 'totalFertile'],
      dtype=

#### Recoding the categorical vars into OneHot

##### Payment Types 1,2,3

In [4]:
aprHospFeat['Payment Typology 1'].unique()[:-1]

array(['Medicare', 'Blue Cross/Blue Shield', 'Private Health Insurance',
       'Self-Pay', 'Medicaid', 'Federal/State/Local/VA',
       'Miscellaneous/Other', 'Department of Corrections',
       'Managed Care, Unspecified', 'Unknown'], dtype=object)

In [5]:
for i in range(len(aprHospFeat['Payment Typology 1'].unique())-1):
    aprHospFeat.loc[(aprHospFeat['Payment Typology 1'] == aprHospFeat['Payment Typology 1'].unique()[i]) | 
                (aprHospFeat['Payment Typology 2'] == aprHospFeat['Payment Typology 1'].unique()[i]) |
                (aprHospFeat['Payment Typology 3'] == aprHospFeat['Payment Typology 1'].unique()[i]), 
                    'payment_'+aprHospFeat['Payment Typology 1'].unique()[i]] = 1

In [6]:
 aprHospFeat.filter(like = 'payment').fillna(0, inplace=True)

/nfshome/yg833/.conda/envs/mgPUI/lib/python3.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


##### Type of Admission

In [7]:
aprHospFeat['Type of Admission'].unique()

array(['Urgent', 'Elective', 'Emergency', 'Newborn', 'Not Available',
       'Trauma', nan], dtype=object)

In [8]:
aprHospFeat = pd.get_dummies(aprHospFeat, columns=['Type of Admission'])

##### Emergency Department Indicator

In [9]:
aprHospFeat['Emergency Department Indicator'].unique()

array(['Y', 'N', nan], dtype=object)

In [10]:
aprHospFeat.loc[aprHospFeat['Emergency Department Indicator'] == 'Y', 'emergencyRoom' ]= 1

In [11]:
aprHospFeat['emergencyRoom'].fillna(0, inplace=True)

##### Hospital Ownership

In [12]:
aprHospFeat = pd.get_dummies(aprHospFeat, columns=['Hospital Ownership'])

##### Rural vs Urban Setting

In [13]:
aprHospFeat = pd.get_dummies(aprHospFeat, columns=['rural_versus_urban'])

In [14]:
aprHospFeat.drop(['Payment Typology 1', 'Payment Typology 2', 'Payment Typology 3', 
                 'Emergency Department Indicator'], 
                axis=1, inplace=True)

#### Feature Engineering & Creating New Variables

##### 1. ratio # insured to # total population

In [15]:
aprHospFeat['ratioInsuredTotalPopulation'] = aprHospFeat['totalInsured']/ aprHospFeat['totalPopulation']

##### 2. ratio # fertile women to total population

In [16]:
aprHospFeat['ratioFertilityTotalPopulation'] = aprHospFeat['totalFertile']/ aprHospFeat['totalPopulation']

##### 3. ratio total charges to total costs

In [17]:
aprHospFeat['ratioChargesCosts'] = aprHospFeat['Total Charges']/ aprHospFeat['Total Costs']

##### 4. charges per day of stay

In [18]:
aprHospFeat.loc[aprHospFeat['Length of Stay']== '120 +', 'days'] = 120

In [19]:
aprHospFeat.loc[aprHospFeat['Length of Stay'] != '120 +', 'days'] = aprHospFeat['Length of Stay']

In [20]:
aprHospFeat['chargesPerDay'] = aprHospFeat['Total Charges']/ pd.to_numeric(aprHospFeat['days'])

##### 5. Number of Payments Used

In [24]:
aprHospFeat['numberPaymentTypes'] = aprHospFeat.filter(like = 'payment').sum(axis=1)

##### 6. 

In [36]:
aprHospFeat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551506 entries, 0 to 2551505
Data columns (total 62 columns):
Unnamed: 0                                                        int64
Health Service Area                                               object
Facility Id                                                       float64
Length of Stay                                                    object
APR DRG Code                                                      float64
APR DRG Description                                               object
APR Severity of Illness Code                                      float64
Total Charges                                                     float64
Total Costs                                                       float64
yr                                                                float64
Provider ID                                                       float64
Hospital overall rating                                           float64
Mort